# Welcome to colaboratory-rdp 👋
[![Documentation](https://img.shields.io/badge/documentation-yes-brightgreen.svg)](https://github.com/BennyExtreme/colaboratory-rdp#readme)
[![Twitter: BennyExtreme\_TW](https://img.shields.io/twitter/follow/BennyExtreme\_TW.svg?style=social)](https://twitter.com/BennyExtreme\_TW)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BennyExtreme/colaboratory-rdp/blob/main/colaboratory-rdp.ipynb)

> Script for Google Colaboratory that allows you to remotely control Google machines via a Chrome Remote Desktop connection, SSH or Remote Shell.

### 🏠 [colaboratory-rdp](https://github.com/BennyExtreme/colaboratory-rdp)

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-bdf00222-74df-8ead-24d7-39fb17c4efa0)


In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.9267729799999955
GPU (s):
0.04388953499999104
GPU speedup over CPU: 66x


In [5]:
#@title **USER CREATOR**
#@markdown > **You must execute this snippet if you want to execute the following ones**

#@markdown > Created users will be placed in the sudo group

#@markdown Enter Username and Password:

import os

Username = "root" #@param {type:"string"}
Password = "toot" #@param {type:"string"}

if Username != "" and Password != "":
    print("Creating user...")

    os.system(f"useradd -m {Username}")
    os.system(f"adduser {Username} sudo")
    os.system(f"echo '{Username}:{Password}' | sudo chpasswd")
    os.system("sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd")

    print(f'User created successfully!\n\nUsername: "{Username}"\nPassword: "{Password}"')
else:
    print("The Username and Password fields cannot be empty!")

Creating user...
User created successfully!

Username: "root"
Password: "toot"


In [6]:
#@title **SSH**

! pip install colab_ssh --upgrade &> /dev/null

#@markdown Setting up the server
#@markdown * Choose a method:
#@markdown   * **Tunshell**: You DON'T need to register an account, reverse shell connection will be created
#@markdown     > For the correct execution of the reverse shell this snippet must remain in execution, otherwise the connection will be interrupted
#@markdown   * **Ngrok**: You MUST register an account, an SSH connection will be created through a tunnel
Method = "Ngrok" #@param ["Tunshell", "Ngrok"]

#@markdown Tunshell setup
#@markdown > Only if you selected **Tunshell** in the Method field!

#@markdown * Open https://tunshell.com/go
#@markdown * Select what you have available on your computer (required to establish a reverse shell connection) in the field on the left
#@markdown * Select "Unix (curl)" or "Unix (wget)" in the field on the right
#@markdown * Click on "Generate session" button
#@markdown * If you selected "This browser" in the left field use the button "Launch Shell" otherwise copy the script snippet on the left and paste it in a terminal of your computer
#@markdown * Copy the script snippet on the left
#@markdown * Paste it here:
Script = "" #@param {type:"string"}

#@markdown Ngrok setup
#@markdown > Only if you selected **Ngrok** in the Method field!

#@markdown * Open https://dashboard.ngrok.com/signup and register a new account
#@markdown   > If you already have one then open https://dashboard.ngrok.com/login and use your existing credentials to access
#@markdown * Open https://dashboard.ngrok.com/auth
#@markdown * Copy the Authtoken
#@markdown * Paste it here:
Authtoken = "29IW4py5VmbRg5rol2phwUtXFMc_5rs2KSDPvbpw4cYgqJuYy" #@param {type:"string"}
#@markdown * Choose your nearest region:
Region = "India" #@param ["United States", "Europe", "Asia/Pacific", "Australia", "South America", "Japan", "India"]

if Region == "United States":
    fixedRegion = "us"
elif Region == "Europe":
    fixedRegion = "eu"
elif Region == "Asia/Pacific":
    fixedRegion = "ap"
elif Region == "Australia":
    fixedRegion = "au"
elif Region == "South America":
    fixedRegion = "sa"
elif Region == "Japan":
    fixedRegion = "jp"
elif Region == "India":
    fixedRegion = "in"


def runTunshell():
    if Script != "":
        print("Starting reverse shell...\nIf the Script didn't contain errors the reverse shell should be ready!")
        os.system(Script)
    else:
        print("The Script field cannot be empty!")

def runNgrok():
    if Authtoken != "":
        print("Starting SSH...")
        from colab_ssh import launch_ssh
        from IPython.display import clear_output

        launch_ssh(Authtoken, password, region=fixedRegion)
        clear_output()
        print("Starting SSH...\nSSH started successfully!\n\nSSH command:")
        print("ssh", user, end='@')
        ! curl -s http://localhost:4040/api/tunnels | python3 -c \
                "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'][6:].replace(':', ' -p '))"
    else:
      print("The Authtoken field cannot be empty!")


try:
    user = Username
    password = Password
except NameError:
    print("No user found, check the first snippet!")

if Method == "Tunshell":
    runTunshell()
elif Method == "Ngrok":
    runNgrok()

Starting SSH...
SSH started successfully!

SSH command:
ssh root@0.tcp.in.ngrok.io -p 11768


In [ ]:
#@title **COLAB CONTROL PANEL**

#@markdown Kill NGROK Tunnel
KILL_NGROK = False #@param {type:'boolean'}

#@markdown Anti AFK
ANTI_AFK = True #@param {type:'boolean'}

if KILL_NGROK:
    print("Killing Ngrok...")
    ! killall ngrok
    print("Ngrok killed successfully!")

if ANTI_AFK:
    print("Anti AFK started successfully!")
    while True:
        continue

Anti AFK started successfully!


## Author

👤 **Benito Alessandrino**

* Website: [Linktree](https://linktr.ee/BennyExtreme)
* Twitter: [@BennyExtreme\_TW](https://twitter.com/BennyExtreme\_TW)
* GitHub: [@BennyExtreme](https://github.com/BennyExtreme)

## 📝 License

Copyright © 2021-2022 [Benito Alessandrino](https://github.com/BennyExtreme).